## PoC парсера сайта kotelock.ru

In [2]:
#начинаем с импорта библиотек
import requests
from requests.auth import HTTPProxyAuth # для проксей
from  bs4 import BeautifulSoup as bs
import csv
from urllib.request import urljoin
#from tqdm import tqdm_notebook
from tqdm.notebook import tqdm
from collections import deque
import pandas as pd
#import os

In [3]:
# Задаем глобальные параметры
headers = {'accept': '*/*',
           'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.66 Safari/537.36'
          }
domen = 'https://kotelock.ru/'
url='https://kotelock.ru/collection/catalog'
filename = 'kotelock_product_links.csv'

Запуск связки от сбора продуктовых категорий до сохранения ссылок на продуктовые страницы в файл

In [12]:
#формируем список продуктовых категорий и скармливаем его пагинатору
temp_list=[]
prodact_cat = get_category_links(headers, domen, url)
for i in prodact_cat:
    a = get_pagination(i, headers) #запуск пагинатора.на одной итерации обрабатывается одна категория
    temp_list.append(a)
#выпрямляем список пагинационных урлов    
grid_links = []
for sublist in temp_list:
    for item in sublist:
        grid_links.append(item)
#сбор продуктовых урлов с грида        
temp_plinks=[]
for gl in grid_links:
    gpl = get_product_links(gl, headers, domen)
    temp_plinks.append(gpl)
#выпрямляем список продуктовых урлов    
product_links=[]
for sublist in temp_plinks:
    for item in sublist:
        product_links.append(item)   
save_products(product_links)
print('DONE!')

<ipython-input-7-444b82df9601>:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(0,count+1), desc='Getting pagination pages', leave=True): #experimental tooling



DONE!


In [5]:
# Функция сбора ссылок на целевые страницы
def get_product_links(url, headers, domen):
    session = requests.Session()
    r = session.get(url, headers=headers)
    soup = bs(r.text, 'lxml')
    links = []
    urls=[]
    if url:
        ul = soup.find('div', class_='main-product-block clearfix').find_all('a', href=True)
        if ul:
            for u in ul:
                if str(u['href'])[0:11] == '/collection':
                    continue
                urls.append(u['href'])
            for i in urls:
                link  = urljoin(domen,i)
                links.append(link)
    return links

In [6]:
# Функция сбора ссылок на основные продуктовые категории
def get_category_links(headers, domen, url):
    session = requests.Session()
    r = session.get(url, headers=headers)
    soup = bs(r.text, 'lxml')
    links = []
    urls=[]
    ul = soup.find_all('a', class_='category-block')
    if ul:
        for u in ul:
             urls.append(u['href'])
    for i in urls:
        link  = urljoin(domen,i)
        links.append(link)
    return links

In [7]:
#  блок сбора урлов пагинации
def get_pagination(url, headers):
    session = requests.Session()
    r = session.get(url, headers=headers)
    pagi=[] 
    pagi.append(url)
    if r.status_code == 200:
        soup = bs(r.content, 'lxml')
            count = int(soup.find('div', class_='pagination-list').find_all('a', title=True)[-1].text)
            for i in tqdm(range(0,count+1), desc='Getting pagination pages', leave=True): #experimental tooling
            #for i in range(count+1):
                u = str(url)+f'?page={i}' #итератор перебора урлов в пагинации
                if u not in pagi:
                    pagi.append(u)
            del pagi[1:3] #удалим нулевую и первую пагинацию
        except:   
            pass
        return pagi

In [4]:
# Функция сохранения файла со ссылками на целевые страницы
def save_products (result):
    with open(filename, 'w', encoding = 'utf-8') as outfile:
        spamwriter = csv.writer(outfile, lineterminator='\n')
        spamwriter.writerow(['Product_Link']) 
        for i in result:
            spamwriter.writerow([i])
            
# Функция загрузки файла со ссылками на целевые страницы
def open_csv(filename):
    results=[]
    with open(filename, 'r') as data:
        data.readline().rstrip()
        for line in data:
            results.append(line[:-1])
    return results

#Функция сохранения результатов парсинга целевых страниц
def save_product_info(products):
    with open('products.csv', 'w', encoding = 'utf-8') as outfile:
        spamwriter = csv.DictWriter(outfile, fieldnames=get_fields_name(products), lineterminator='\n')    
        spamwriter.writeheader() #записывет заголовки столбцов равные ключам словаря
        spamwriter.writerows(products)
        
# Вспом.функция для сбора названия полей с словарях с продуктами для формирования заголовка в CSV (Без сортировки)
def get_field_name(products):
    field_names = set()
    for i in products:
        for k in i:
            field_names.add(k)
    return field_names

#версия 2 с сортированными названиями заголовков (текущая) Использовать её!
def get_fields_name(products):
    headerz=[]
    field_names = set()
    for i in products:
        for k in i:
            field_names.add(k)
    for i in field_names:
        if i=='1_title':
            headerz.append(i)
        if i=='2_articul':
            headerz.append(i)
        if i=='3_price':
            headerz.append(i)
        if i=='4_consistation':
            headerz.append(i)
        #if i=='6_volume':
            #headerz.append(i)
        if i=='5_weight/volume':
            headerz.append(i)
        if i=='7_manufacturer':
            headerz.append(i)
        if i=='8_stock':
            headerz.append(i)
        if i=='91_url':
            headerz.append(i)
        if i=='92_image':
            headerz.append(i)
        if i=='93_description':
            headerz.append(i)
        headerz.sort()            
    return headerz

#функция проверки уже спарсенных урлов (чтобы не запускать по ним парсинг повторно)
def get_visited_urls():
    try:
        file = 'products.csv'
        if file:
            df = pd.read_csv(file, encoding = 'utf-8')
            urls_done = df['91_url'].tolist()
            visited = set(urls_done)
            return visited, df
    except FileNotFoundError:
        df=pd.DataFrame()
        visited=set()
        return visited, df # если файла не существует, то вернет путой сет и список

# Функция объединения прошлых и текущих результатов в 1 общий файл
def add_previous_part(df):
    if df.shape[0] != 0:
        file = 'C:/Users/Vladimir/Coding/Notebooks/Парсинг/products.csv'
        df.to_csv(file, mode='a', index=False, encoding = 'utf-8', header=False)
    else:
        pass   

Версия 1  - самая простая. Она просто сохраняет итоговый результат в файл. Как опцию,можно включить сохранение
в SQLlite базу. Для этого надо раскоментить соотв. строчку. Главнй цикл реализован с помощью tdqm.

Версия 2  - уже более сложная. В ней реализована функция промежуточного сохранения в случае,
если на какой-то итерации происходит проблема с супом (в условии прописано if soup/else, хотя можно дописать
что if soup else if soup = None soup/else. Т.е вроде суп есть, но он пустой)
В связи с тем, что предполагается что у нас есть некие промежуточные результаты в версию 2 заложен механизм
проверки уже спарсеных страниц и такие страницы повторно не парсятся. 
Новые результаты объеденяются со предыдущими результатами в том же файле.
Есть некая особенность, если запустить 2 раза парсинг подряд, то файл products будет без хедера. но со всем содержимым.
Это потому что  при аппенде датафрейма отключены хедеры.

Версия 3 - такая же, как и версия 2 +  я пробовал обработать эксепшены, но получилось не очень. Сам код без ошибок.
Но я тестировал просто врубая на ноуте самолетный режим, эмулируя экстренную потерю соединения. Я занес в эксепшн 2 ошибки, которые вылезли при вкл. самолета, но код их не обрабатывает. все равно выбивает на эксепшн.  Понятно, что если сервер
будет банить, скорее всего будет какая-нб 400-я или 500-я ошибка. Еще в этой версии я отключил tdqm т.к. он выглядит крипово
на while цикле с очередью (decue) - прогресс бар не работает
Вместо него просто прописал строковый output +добавил итератор по циклу. Стало приятнее на глаз.

In [14]:
get_info3(headers, filename)  # - запуск главного модуля парсера

Обрабатываю ссылку № 1
Обрабатываю ссылку № 2
Часть тегов не обнаружена. Двигаюсь дальше

Обрабатываю ссылку № 3
Часть тегов не обнаружена. Двигаюсь дальше

Обрабатываю ссылку № 4
Часть тегов не обнаружена. Двигаюсь дальше

Обрабатываю ссылку № 5
Часть тегов не обнаружена. Двигаюсь дальше

Обрабатываю ссылку № 6
Часть тегов не обнаружена. Двигаюсь дальше

Обрабатываю ссылку № 7
Часть тегов не обнаружена. Двигаюсь дальше

Обрабатываю ссылку № 8
Часть тегов не обнаружена. Двигаюсь дальше

Обрабатываю ссылку № 9
Часть тегов не обнаружена. Двигаюсь дальше

Обрабатываю ссылку № 10
Часть тегов не обнаружена. Двигаюсь дальше

Обрабатываю ссылку № 11
Часть тегов не обнаружена. Двигаюсь дальше

DONE!


In [6]:
# ВЕРСИЯ 1
def get_info(headers, filename):
    results = open_csv(filename)
    products=[]
    session = requests.Session()
    #for url in results:
    for url in tqdm(results, desc='Getting info from product page url', leave=False): #experimental tooling   
        params={}
        r = session.get(url, headers=headers)
        soup = bs(r.text, 'lxml')
        if soup:
            try:
                params['91_url']=url
                j=soup.find('div', class_='my_recipe_image-box').find_next('img')['src']
                if j:
                    params['92_image']=j
                a=soup.find('span', class_='product-title').text
                if a:
                    params['1_title']=a
                i=soup.find('div', class_='product-desc', p='').text
                if i: 
                    params['93_description']=i
                b=soup.find('span', style='white-space:nowrap;').text
                if b:
                    params['2_articul']=b.strip()
                c=soup.find('span', class_='this_product_price itemPrice').text.lstrip('Цена:').strip()
                if c:
                    params['3_price']=c 
                d=soup.find('span', class_='itemNal product-option-isset').text    
                if d: 
                    params['8_stock']=d
                e=soup.find('span', class_='itemLbl product-option-weight', span='').find_next_sibling('span').text.strip()
                if e: 
                    params['5_weight/volume']=e
                f=soup.find('div', class_='itemVal', span='').text.lstrip('Объем:').strip()
                #if f: 
                    #params['6_volume/weight']=f
               #g=soup.find('span', class_='itemLbl product-option-manufacturer', span='').find_next_sibling('span').text
                if g: 
                    params['7_manufacturer']=g
                h=soup.find('span', class_='itemLbl product-option-consist', span='').find_next_sibling('span').text
                if h: 
                    params['4_consistation']=h
                i=soup.find('div', class_='product-desc', p='').text
                if i: 
                    params['93_description']=i
                products.append(params)
            except:
                products.append(params)
                print('Часть тегов не обнаружена. Двигаюсь дальше')
                pass
        else:
            print('Проблемы с супом')
            save_product_info(products) # промежуточное сохранение на случай бана
            return False
    save_product_info(products) # для сохранения в CSV
    #save_to_sqlite(database_path, products) # раскоментить для сохранения в SQLite базу
    #return products
    print('DONE!')

In [4]:
# ВЕРСИЯ 2 С ПРОМЕЖУТОЧНЫМ СОХРАНЕНИЕМ
def get_info2(headers, filename):
    global df
    results = open_csv(filename)
    queue = deque()
    queue.extend(results)
    visited = get_visited_urls()[0]
    df = get_visited_urls()[1]
    session = requests.Session()
    products = [] 
    #for url in queue:
    #for url in tqdm(queue, desc='Getting info from product page url', leave=False): #experimental tooling 
    while tqdm(queue, desc='Getting info from product page url', leave=False): #experimental tooling    
        url = queue.popleft()
        if url in visited:
            continue   
        params={} 
        r = session.get(url, headers=headers)
        soup = bs(r.text, 'lxml')
        if soup:
            visited.add(url) 
            try:
                params['91_url']=url
                j=soup.find('div', class_='my_recipe_image-box').find_next('img')['src']
                if j:
                    params['92_image']=j
                a=soup.find('span', class_='product-title').text
                if a:
                    params['1_title']=a
                i=soup.find('div', class_='product-desc', p='').text
                if i: 
                    params['93_description']=i
                b=soup.find('span', style='white-space:nowrap;').text
                if b:
                    params['2_articul']=b.strip()
                c=soup.find('span', class_='this_product_price itemPrice').text.lstrip('Цена:').strip()
                if c:
                    params['3_price']=c 
                d=soup.find('span', class_='itemNal product-option-isset').text    
                if d: 
                    params['8_stock']=d
                e=soup.find('span', class_='itemLbl product-option-weight', span='').find_next_sibling('span').text.strip()
                if e: 
                    params['5_weight/volume']=e
                f=soup.find('div', class_='itemVal', span='').text.lstrip('Объем:').strip()
                #if f: 
                    #params['6_volume/weight']=f
               #g=soup.find('span', class_='itemLbl product-option-manufacturer', span='').find_next_sibling('span').text
                if g: 
                    params['7_manufacturer']=g
                h=soup.find('span', class_='itemLbl product-option-consist', span='').find_next_sibling('span').text
                if h: 
                    params['4_consistation']=h
                i=soup.find('div', class_='product-desc', p='').text
                if i: 
                    params['93_description']=i
                products.append(params)
            except:
                products.append(params)
                print('Часть тегов не обнаружена. Двигаюсь дальше')
                pass
        else:
            print('Проблемы с супом')
            save_product_info(products)
            return False
    save_product_info(products)
    add_previous_part(df) 
    #save_to_sqlite(database_path, products) # для сохранения в SQLite базу
    #return products # только для теста sql3
    print('DONE!')  

In [12]:
# ВЕРСИЯ 3 С ПРОМЕЖУТОЧНЫМ СОХРАНЕНИЕМ И ОБРАБОТКОЙ ЭКСЕПШЕНОВ
def get_info3(headers, filename):
    ConnectErrs  = (TimeoutError, ConnectionError)
    global df
    results = open_csv(filename)
    queue = deque()
    queue.extend(results)
    visited = get_visited_urls()[0]
    df = get_visited_urls()[1]
    session = requests.Session()
    products = []
    Count=0
    #for url in queue:
    #for url in tqdm(queue, desc='Getting info from product page url', leave=False): #не работает с изменяемой очередью
    #while tqdm(queue, desc='Getting info from product page url', leave=True): # отключил т.к выглядит жутко
    while queue:
        Count=Count+1
        msg = f'Обрабатываю ссылку № {Count}'
        print(msg)
        url = queue.popleft()
        if url in visited:
            continue # начнет цикл while заново     
        params={} 
        try:
            r = session.get(url, headers=headers)
            soup = bs(r.text, 'lxml')
            if soup:
                visited.add(url) #добавляем в посещенные, только когда получим суп. т.е. страница доступна
                try:
                    params['91_url']=url
                    j=soup.find('div', class_='my_recipe_image-box').find_next('img')['src']
                    if j:
                        params['92_image']=j
                    a=soup.find('span', class_='product-title').text
                    if a:
                        params['1_title']=a
                    i=soup.find('div', class_='product-desc', p='').text
                    if i: 
                        params['93_description']=i
                    b=soup.find('span', style='white-space:nowrap;').text
                    if b:
                        params['2_articul']=b.strip()
                    c=soup.find('span', class_='this_product_price itemPrice').text.lstrip('Цена:').strip()
                    if c:
                        params['3_price']=c 
                    d=soup.find('span', class_='itemNal product-option-isset').text    
                    if d: 
                        params['8_stock']=d
                    e=soup.find('span', class_='itemLbl product-option-weight', span='').find_next_sibling('span').text.strip()
                    if e: 
                        params['5_weight/volume']=e
                    f=soup.find('div', class_='itemVal', span='').text.lstrip('Объем:').strip()
                    #if f: 
                        #params['6_volume/weight']=f
                   #g=soup.find('span', class_='itemLbl product-option-manufacturer', span='').find_next_sibling('span').text
                    if g: 
                        params['7_manufacturer']=g
                    h=soup.find('span', class_='itemLbl product-option-consist', span='').find_next_sibling('span').text
                    if h: 
                        params['4_consistation']=h
                    i=soup.find('div', class_='product-desc', p='').text
                    if i: 
                        params['93_description']=i
                    products.append(params)
                except:
                    products.append(params)
                    print('Часть тегов не обнаружена. Двигаюсь дальше\n')
                    pass
            else:
                print('Проблемы с супом')
                save_product_info(products)# промежуточное сохранение на случай если суп не обнаружен
                return False
        except ConnectErrs :
            save_product_info(products) #запуск в случае срабатывания эксепшена
            add_previous_part(df) #запуск в случае срабатывания эксепшена
            print('Аварийное завершение работы')
    save_product_info(products) #  для сохранения в CSV
    add_previous_part(df) # перезапись предыдущих результатов в случае перезапуска парсера
    print('DONE!')  

In [125]:
# запуск сохранения в SQL3-базу
save_to_sqlite(database_path, rows)

In [90]:
database_path = '../SQL-light/kotelock.db3'
#rows = get_info(headers, filename)
#sqlite_insert = "INSERT OR REPLACE INTO kotelock_tab (articul,title,price,weight,stock,url,image,description) VALUES (?,?,?,?,?,?,?,?)"
sqlite_insert = "INSERT INTO kotelock_tab (articul,title,price,weight,stock,url,image,description) VALUES (?,?,?,?,?,?,?,?)"


#F0 - main block to save in SQL3
def save_to_sqlite(database_path, rows):
    global connection
    connection = __connect(database_path) #F1
    __create_table() #F2
    for row in rows:
        __save_row(row) #F3 row - это 1 строка нашего словарика
    __close_connection() #F4
    
#F1
def __connect(database):
    conn = sqlite3.connect(database).cursor()  
    return conn

#F2
def __create_table():
    connection.execute("""CREATE TABLE IF NOT EXISTS kotelock_tab (
                                articul TEXT PRIMARY KEY,
                                title TEXT NOT NULL,
                                price TEXT,
                                weight TEXT,
                                stock TEXT,
                                url TEXT,
                                image TEXT,
                                description TEXT)
                        """)

#F3
def __save_row(row):
    connection.execute(sqlite_insert, (
    row.get('2_articul'),    
    row.get('1_title'),
    row.get('3_price'),
    row.get('5_weight/volume'),
    row.get('8_stock'),
    row.get('91_url'),
    row.get('92_image'),
    row.get('93_description')
    ))
    
#F4
def __close_connection():
    if connection:
            connection.close()